# Healthcare Natural Language API

In [8]:
import requests
import subprocess
import json 

sample_text = "Course in Hospital:  Mr. Johnson arrived in the ER from nursing home with a three-day history of worsening shortness of breath, yellow-green sputum, and increased sputum production. He was subsequently diagnosed with a COPD exacerbation and was satting at 84% on 4L O2 by nasal prongs. Medical presciptions : TAB PARACIP 500 MG two TABLETS PER ORAL THRICE DAILY AFTER FOOD FOR 5 DAYS INJ. AUGMENTIN 1.2 GM, INTRAVENOUS, THREE TIMES A DAY X 4 DAYS"

def get_access_token():
    result = subprocess.run(['gcloud', 'auth', 'print-access-token'], capture_output=True, text=True)
    return result.stdout.strip()

access_token = get_access_token()

headers = {
    "Authorization": f"Bearer {access_token}",  
    "Content-Type": "application/json"
}

url = "https://healthcare.googleapis.com/v1/projects/{project_id}/locations/us-central1/services/nlp:analyzeEntities"

data = {
  "documentContent": sample_text
}

response = requests.post(url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    print("Request successful!")
    # Process the response data here
else:
    print("Request failed:", response.text) 

Request successful!


In [ ]:
response.json()

# Medpalm

In [ ]:
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

# The AI Platform services require regional API endpoints.
client_options = {"api_endpoint": "us-central1-aiplatform.googleapis.com"}
# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = aiplatform.gapic.PredictionServiceClient(
    client_options=client_options
)
instance_dict = {
    "content": "You are a medical expert and you have to all extract Medicine Name, dosage, frequency, duration from the following text in the format\n[Medicine Name, Medicine strength, Medicine Dose, Medicine frequency, Medicine duration].\nif medicine attribute is not found fill None. Output should be strictly a list containing 5 items.\nDo not include medicine route like INTRAVENOUS, SUBCUTANEOUS in output\n\nExample of\nMedical strength = ['10mg', '90MG', '40 MG']\nDosage = ['ONE TABLET', 'two doses', '3 TSF', '2 PUFFS', '1 TAB'] dosage.\nFrequency = ['THRICE DAILY', 'ONCE AT BED TIME', 'TWICE A DAY', 'ONCE A DAY']\nDuration = ['21 days / 28 days', '15 days', 'one week']\n\nInput : \"1. TAB PARACIP 500 MG two TABLETS PER ORAL THRICE DAILY AFTER FOOD FOR 5 DAYS (PAIN)\n2. TAB PAN 40 MG ONE TABLET PER ORAL ONCE DAILY BEFORE BREAKFAST FOR 7 DAYS (ACIDITY)\"\nExpected output : [['TAB PAN', '40 MG','ONE TABLET', 'ONCE DAILY', '7 DAYS'], ['TAB PARACIP ', '500 MG','two TABLETS','THRICE DAILY', '5 DAYS']]"
}
instance = json_format.ParseDict(instance_dict, Value())
instances = [instance]
parameters_dict = {
    "maxOutputTokens": 500,
    "temperature": 0.2,
    "topP": 0.8,
    "topK": 40
}
parameters = json_format.ParseDict(parameters_dict, Value())
response = client.predict(
    endpoint="projects/{project_id}/locations/us-central1/publishers/google/models/medpalm2@latest", instances=instances, parameters=parameters
)
print("response")
predictions = response.predictions
for prediction in predictions:
  print(" prediction:", dict(prediction))

# Medspacy

In [ ]:
!pip install medspacy
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import medspacy

nlp = medspacy.load("en_core_web_sm") # Load a pre-trained MedSpaCy model 
text = "Patient reports headache and nausea. Denies fever. Prescribed ibuprofen 200mg."
doc = nlp(text)

for entity in doc.ents:
    print(entity.text, entity.label_)
